In [1]:
from math import nan
from future.utils import iteritems
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#from utils import get_data, generate_output, guess_human, seed_sequence, get_embeddings, find_closest
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt


Using TensorFlow backend.


In [2]:
data = pd.read_csv("ner_dataset.csv", encoding= 'latin1') 
data = data.fillna(method="ffill")
data.head(672)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
667,Sentence: 29,Muslim,NNP,B-org
668,Sentence: 29,Brotherhood,NNP,I-org
669,Sentence: 29,as,IN,O
670,Sentence: 29,parts,NNS,O


In [6]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
nrofWords = len(words)

ner_tags = list(set(data["Tag"].values))
nrofTags = len(tags)
print(nrofWords, nrofTags)

35179 17


In [4]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["Word"].values.tolist(),
                                                        s["Tag"].values.tolist())]
        self.grouped = self.dataset.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
          
getter = SentenceGetter(data)
sentences = getter.sentences

In [8]:
word_index = {w: i +1 for i, w in enumerate(words)}
tag_index = {t: i for i, t in enumerate(ner_tags)}
idx2tag = {v: k for k, v in iteritems(tag_index)}

In [9]:
word_index["universe"]

30055

In [11]:
maxlen = max([len(s) for s in sentences])
word_embedding_size = 17

X = [[word_index[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=nrofWords - 1)

y = [[tag_index[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag_index["O"])
y = [to_categorical(i, num_classes=nrofTags) for i in y]

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Simple RNN model

In [31]:
input = Input(shape=(maxlen,))

model_1 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=nrofWords+1, output_dim=20, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(8, return_sequences=True)),
    tf.keras.layers.Dense(nrofTags, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
])



In [32]:
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model_1.summary()

history_1 = model_1.fit(X_train, np.array(y_train), batch_size=32, epochs=10,
                    validation_split=0.1, verbose=1)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 104, 20)           703600    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 104, 16)           464       
_________________________________________________________________
dense_3 (Dense)              (None, 104, 17)           289       
_________________________________________________________________
dropout_3 (Dropout)          (None, 104, 17)           0         
Total params: 704,353
Trainable params: 704,353
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1080/1080 [==============================] - 44s 41ms/step - loss: 0.5605 - accuracy: 0.2021 - val_loss: 0.3713 - val_accuracy: 0.9683
Epoch 2/10
1080/1080 [==============================] - 39s 37ms/step - loss: 0.4262 - accuracy: 0.4

In [35]:
test_pred_1 = model_1.predict(X_test, verbose=1)

idx2tag = {i: w for w, i in tag_index.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred_1)
test_labels = pred2label(y_test)

300/300 [==============================] - 3s 9ms/step


In [36]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))
print(classification_report(test_labels, pred_labels))

F1-score: 24.8%
           precision    recall  f1-score   support

      per       0.00      0.00      0.00      3376
      org       0.01      0.00      0.00      4046
      gpe       0.00      0.00      0.00      3059
      geo       0.36      0.32      0.34      7538
      tim       0.28      0.52      0.36      4066
      eve       0.00      0.00      0.00        69
      nat       0.00      0.00      0.00        42
      art       0.00      0.00      0.00        65

micro avg       0.31      0.21      0.25     22261
macro avg       0.18      0.21      0.18     22261



In [37]:
i = 1928
p = model_1.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_test[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_test[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-1], ner_tags[t], ner_tags[pred]))


Word           ||True ||Pred
Qantas         : B-org O
chief          : O     O
executive      : O     O
Alan           : B-per O
Joyce          : I-per O
joined         : O     O
the            : O     O
first          : O     O
leg            : O     O
of             : O     O
the            : O     O
flight         : O     O
to             : O     O
London         : B-geo B-geo
via            : O     O
Singapore      : B-geo O
.              : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : 

# LSTM model

In [18]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=nrofWords+1, output_dim=word_embedding_size, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Dense(nrofTags, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
])

In [19]:
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [20]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 104, 17)           598060    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 104, 128)          41984     
_________________________________________________________________
dense_1 (Dense)              (None, 104, 17)           2193      
_________________________________________________________________
dropout_1 (Dropout)          (None, 104, 17)           0         
Total params: 642,237
Trainable params: 642,237
Non-trainable params: 0
_________________________________________________________________


In [21]:
history_2 = model_2.fit(X_train, np.array(y_train), batch_size=32, epochs=30,
                    validation_split=0.5, verbose=1)


Epoch 1/30
600/600 [==============================] - 54s 91ms/step - loss: 0.4553 - accuracy: 0.4892 - val_loss: 0.3191 - val_accuracy: 0.9679
Epoch 2/30
600/600 [==============================] - 53s 89ms/step - loss: 0.4143 - accuracy: 0.4922 - val_loss: 0.3178 - val_accuracy: 0.9679
Epoch 3/30
600/600 [==============================] - 48s 81ms/step - loss: 0.4140 - accuracy: 0.4919 - val_loss: 0.3175 - val_accuracy: 0.9679
Epoch 4/30
600/600 [==============================] - 53s 88ms/step - loss: 0.4139 - accuracy: 0.4920 - val_loss: 0.3174 - val_accuracy: 0.9679
Epoch 5/30
600/600 [==============================] - 47s 78ms/step - loss: 0.4138 - accuracy: 0.4917 - val_loss: 0.3173 - val_accuracy: 0.9679
Epoch 6/30
600/600 [==============================] - 47s 78ms/step - loss: 0.4138 - accuracy: 0.4919 - val_loss: 0.3173 - val_accuracy: 0.9679
Epoch 7/30
600/600 [==============================] - 47s 79ms/step - loss: 0.4138 - accuracy: 0.4919 - val_loss: 0.3172 - val_accuracy:

In [22]:
test_pred_2 = model_2.predict(X_test, verbose=1)

#idx2tag = {i: w for w, i in index_tag.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred_2)
test_labels = pred2label(y_test)


300/300 [==============================] - 4s 15ms/step


In [23]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

F1-score: 11.0%


In [24]:
print(classification_report(test_labels, pred_labels))

           precision    recall  f1-score   support

      per       0.14      0.61      0.22      3376
      org       0.00      0.00      0.00      4046
      gpe       0.00      0.00      0.00      3059
      geo       0.00      0.00      0.00      7538
      tim       0.00      0.00      0.00      4066
      eve       0.00      0.00      0.00        69
      nat       0.00      0.00      0.00        42
      art       0.00      0.00      0.00        65

micro avg       0.14      0.09      0.11     22261
macro avg       0.02      0.09      0.03     22261



In [25]:
i = 2007
p = model_1.predict(np.array([X_test[i]]))
p = np.argmax(p, axis=-1)
true = np.argmax(y_test[i], -1)
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")
for w, t, pred in zip(X_test[i], true, p[0]):
    if w != 0:
        print("{:15}: {:5} {}".format(words[w-1], ner_tags[t], ner_tags[pred]))


Word           ||True ||Pred
In             : O     O
it             : O     O
,              : O     O
Zawahiri       : B-per O
appeared       : O     O
in             : O     O
front          : O     O
of             : O     O
a              : O     O
portrait       : O     O
of             : O     O
Zarqawi        : B-geo B-geo
.              : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : O     O
physicist      : 